In [29]:
import pandas as pd
import MeCab
import fitz
import re
import sys
import itertools
from collections import Counter


In [69]:

text = "管理籍"
text="そして、このしくみの対象となる当社グループ会社（国内）は、みずから生み出しまたは譲り受けた特許等を、PHDへの法的名義集約後も管理し（「管理籍」の保持）、当該特許等の費用を負担し、収益を収受します。"
text="現在のソニーでは、Purposeの推進に向け、これら価値創造のドライバーに対する経営資源の重点配分や、ドライバーのさらなる融合に注力しています。事業間連携は定着しつつあり、コンテンツIPのグループ内での展開によるIP価値向上や、事業を横断したテクノロジーの活用プロジェクトが増加しています。"
text="　「感動」を創り、届けるため、感動クリエイション、すなわちコンテンツIPの強化に取り組んでいます。\nコンテンツそのものへの投資に加え、クリエイターに近づき、感動を創る力にも投資しています。過去5年のコンテンツIPへの投資額は約1兆円です。"
#text="…そして、このしくみの対象となる当社グループ会社（国内）は、みずから生み出しまたは譲り受けた特許等を、PHDへの法的名義集約後も管理し（「管理籍」の保持）、当該特許等の費用を負担し、収益を収受します。"
#text = "貴社の記者が汽車で帰社した。"

In [70]:
def talknizer_bare(text):
    
#    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t") for l in parsed_lines]#原形を取得

    return token_list

def token_bare(text):
    
    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
#    path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    return token_list


def token_with_userdic(text):
    
    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    return token_list

def token_noun(text):
    
    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
#    path1 = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    mecab = MeCab.Tagger(path1)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    #--------------------------------------------
    #品詞区分による絞り込み：Chasenの品詞区分を個別に指定する場合
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    return token_list

def token_noun_with_userdic(text):
    
    path1 = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    path2 = " -u /work_dir/userdic/user.dic"
    mecab = MeCab.Tagger(path1+path2)
    parsed_lines = mecab.parse(text).split("\n")[:-2]
    
    token_list = [l.split("\t")[2] for l in parsed_lines]#原形を取得

    #--------------------------------------------
    #品詞区分による絞り込み：Chasenの品詞区分を個別に指定する場合
    #品詞区分の取得（リスト形式）
    pos = [l.split('\t')[3] for l in parsed_lines]

    #抽出する品詞区分の定義（完全一致で指定）
    target_pos = ['名詞-一般',
                  '名詞-固有名詞-一般',
                  '名詞-固有名詞-人名-一般',
                  '名詞-固有名詞-人名-姓',
                  '名詞-固有名詞-人名-名',
                  '名詞-固有名詞-組織',
                  '名詞-固有名詞-地域-一般',
                  '名詞-固有名詞-地域-国',
                  '名詞-代名詞-一般',
                  '名詞-代名詞-縮約',
                  '名詞-副詞可能',
                  '名詞-サ変接続',
                  '名詞-形容動詞語幹',
                  '名詞-ナイ形容詞語幹'
                 ]
    #--------------------------------------------
    #形態素と品詞区分のリストをペアにしてタプルリスト化、該当する品詞区分の形態素のみリストに出力
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    return token_list



In [98]:
#集約したテキストを「。」で区切り、形態素解析（共起分析用）
sentences = [token_noun_with_userdic(sentence) for sentence in re.split("。", text)]
#sentences = [token_noun(sentence) for sentence in re.split("。", text)]

In [99]:
print(sentences)

[['感動', '感動', 'Creation', 'コンテンツIP', '強化'], ['コンテンツ', 'そのもの', '投資', 'Creator', '感動', '力', '投資'], ['過去', '5年', 'コンテンツIP', '投資額', '1兆円'], []]


In [100]:
#各文中の、形態素組み合わせを作る
sentences_combs = [list(itertools.combinations(sentence,2)) for sentence in sentences]
#print(sentences_combs[0])

#組み合わせた2つの形態素の並びをソート
words_combs = [[tuple(sorted(words)) for words in sentence] for sentence in sentences_combs]
#print(words_combs[0][:30])
target_combs = []
for words_comb in words_combs:
    target_combs.extend(words_comb)
ct = Counter(target_combs)
#print(ct)
df = pd.DataFrame([{"1番目" : i[0][0], "2番目": i[0][1], "count":i[1]} for i in ct.most_common()])
#df.head(30)


In [101]:
print(talknizer_bare(text))

[['\u3000', '記号,空白,*,*,*,*,\u3000,\u3000,\u3000'], ['「', '記号,括弧開,*,*,*,*,「,「,「'], ['感動', '名詞,サ変接続,*,*,*,*,感動,カンドウ,カンドー'], ['」', '記号,括弧閉,*,*,*,*,」,」,」'], ['を', '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ'], ['創り', '動詞,自立,*,*,五段・ラ行,連用形,創る,ツクリ,ツクリ'], ['、', '記号,読点,*,*,*,*,、,、,、'], ['届ける', '動詞,自立,*,*,一段,基本形,届ける,トドケル,トドケル'], ['ため', '名詞,非自立,副詞可能,*,*,*,ため,タメ,タメ'], ['、', '記号,読点,*,*,*,*,、,、,、'], ['感動', '名詞,サ変接続,*,*,*,*,感動,カンドウ,カンドー'], ['クリエイション', '名詞,固有名詞,一般,*,*,*,Creation,クリエイション,クリエイション'], ['、', '記号,読点,*,*,*,*,、,、,、'], ['すなわち', '接続詞,*,*,*,*,*,すなわち,スナワチ,スナワチ'], ['コンテンツ', '名詞,一般,*,*,*,*,コンテンツ,コンテンツ,コンテンツ'], ['IP', '名詞,固有名詞,一般,*,*,*,IP,アイピイ,アイピー'], ['の', '助詞,連体化,*,*,*,*,の,ノ,ノ'], ['強化', '名詞,サ変接続,*,*,*,*,強化,キョウカ,キョーカ'], ['に', '助詞,格助詞,一般,*,*,*,に,ニ,ニ'], ['取り組ん', '動詞,自立,*,*,五段・マ行,連用タ接続,取り組む,トリクン,トリクン'], ['で', '助詞,接続助詞,*,*,*,*,で,デ,デ'], ['い', '動詞,非自立,*,*,一段,連用形,いる,イ,イ'], ['ます', '助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス'], ['。', '記号,句点,*,*,*,*,。,。,。'], ['コンテンツ', '名詞,一般,*,*,*,*,コンテンツ,コンテンツ,コンテンツ'], ['そのもの', '名詞,一般,*,*,*,*,そのもの,ソノモノ,ソノモノ'

In [102]:
print(token_bare(text))

['\u3000', '「', '感動', '」', 'を', '創る', '、', '届ける', 'ため', '、', '感動', 'Creation', '、', 'すなわち', 'コンテンツ', 'IP', 'の', '強化', 'に', '取り組む', 'で', 'いる', 'ます', '。', 'コンテンツ', 'そのもの', 'へ', 'の', '投資', 'に', '加える', '、', 'Creator', 'に', '近づく', '、', '感動', 'を', '創る', '力', 'に', 'も', '投資', 'する', 'て', 'いる', 'ます', '。', '過去', '5年', 'の', 'コンテンツ', 'IP', 'へ', 'の', '投資額', 'は', '約', '1兆円', 'です', '。']


In [103]:
print(token_with_userdic(text))

['\u3000', '「', '感動', '」', 'を', '創る', '、', '届ける', 'ため', '、', '感動', 'Creation', '、', 'すなわち', 'コンテンツIP', 'の', '強化', 'に', '取り組む', 'で', 'いる', 'ます', '。', 'コンテンツ', 'そのもの', 'へ', 'の', '投資', 'に', '加える', '、', 'Creator', 'に', '近づく', '、', '感動', 'を', '創る', '力', 'に', 'も', '投資', 'する', 'て', 'いる', 'ます', '。', '過去', '5年', 'の', 'コンテンツIP', 'へ', 'の', '投資額', 'は', '約', '1兆円', 'です', '。']


In [104]:
print(token_noun(text))

['感動', '感動', 'Creation', 'コンテンツ', 'IP', '強化', 'コンテンツ', 'そのもの', '投資', 'Creator', '感動', '力', '投資', '過去', '5年', 'コンテンツ', 'IP', '投資額', '1兆円']


In [105]:
print(text)
print("--------")
print(token_noun(text))
print("--------")
print(token_noun_with_userdic(text))

　「感動」を創り、届けるため、感動クリエイション、すなわちコンテンツIPの強化に取り組んでいます。
コンテンツそのものへの投資に加え、クリエイターに近づき、感動を創る力にも投資しています。過去5年のコンテンツIPへの投資額は約1兆円です。
--------
['感動', '感動', 'Creation', 'コンテンツ', 'IP', '強化', 'コンテンツ', 'そのもの', '投資', 'Creator', '感動', '力', '投資', '過去', '5年', 'コンテンツ', 'IP', '投資額', '1兆円']
--------
['感動', '感動', 'Creation', 'コンテンツIP', '強化', 'コンテンツ', 'そのもの', '投資', 'Creator', '感動', '力', '投資', '過去', '5年', 'コンテンツIP', '投資額', '1兆円']


In [106]:
df.head(20)

,1番目,2番目,count
0,Creation,感動,2
1,コンテンツIP,感動,2
2,強化,感動,2
3,コンテンツ,投資,2
4,そのもの,投資,2
5,Creator,投資,2
6,感動,投資,2
7,力,投資,2
8,感動,感動,1
9,Creation,コンテンツIP,1
